<font size="5">Section 3: Panel data</font>

## Housekeeping and Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
import scipy 
import linearmodels.panel as lmp

%matplotlib inline

We use data from the ENIA firm panel survey. Variable description follows:  


- *ID*: firm unique identifier  
- *year*: survey year  
- *tamano*: 1 large, 2 medium, 3 small, 4 micro  
- *sales*: sales (in log of 1,000 CLP)  
- *age*: firm age at time of survey  
- *foreign*: non-domestic firm (binary)  
- *export*: production for export (binary)  
- *workers*: log of number of workers  
- *fomento*: firm receives public incentives (binary)  
- *iyd*: firm does I+D (binary)  
- *impuestos*: taxes (in million US)  
- *utilidades*: firm revenue (in million US)  

For purposes of this analysis we will consider tamano as a continous value (moving from largest to smallest as number increases).

In [9]:
# enia data
charls = pd.read_csv('../data/charls.csv')
charls.dropna(inplace=True)
charls.drop(charls.index[10057:34371], inplace=True)
charls.reset_index(drop=True, inplace=True)

charls.head()

,cesd,child,drinkly,female,hrsusu,hsize,inid,intmonth,married,retired,schadj,urban,wave,wealth,age
0,6,2,0.None,1,0.0,4,1.010410e+10,7,1,0,0,0,1,-5800.0,46
1,7,2,0.None,1,49.0,4,1.010410e+10,7,1,0,0,0,2,100.0,46
2,5,2,0.None,1,56.0,7,1.010410e+10,8,1,0,0,0,3,-59970.0,46
3,0,2,1.Yes,0,63.0,4,1.010410e+10,7,1,0,4,0,1,-5800.0,48
4,5,2,1.Yes,0,49.0,4,1.010410e+10,7,1,0,4,0,2,100.0,48


In [16]:
#variable construction
X=charls[['child','hrsusu','hsize','retired','wealth','age']]
Xm=(X.groupby(charls['inid']).transform('mean'))
Xid=charls[['inid','wave','cesd','child','hrsusu','hsize','retired','wealth','age']]
Xc=pd.DataFrame(np.c_[Xid, Xm], columns=['inid','wave','cesd','child','hrsusu','hsize','retired','wealth','age','mchild','mhrsusu','mhsize','mretired','mwealth','mage'])

#set panel structure
Xc = Xc.set_index(["inid","wave"])
Xc.describe()

,cesd,child,hrsusu,hsize,retired,wealth,age,mchild,mhrsusu,mhsize,mretired,mwealth,mage
count,10057.000000,10057.000000,10057.000000,10057.000000,10057.000000,1.005700e+04,10057.000000,10057.000000,10057.000000,10057.000000,10057.000000,1.005700e+04,10057.000000
mean,8.866759,2.769116,27.967634,3.652083,0.268569,1.019597e+04,58.225912,2.769116,27.967634,3.652083,0.268569,1.019597e+04,58.225912
std,6.288884,1.436119,27.248319,1.784247,0.443237,9.944435e+04,9.236036,1.361229,21.270696,1.459669,0.365710,6.278602e+04,9.236036
min,0.000000,0.000000,0.000000,1.000000,0.000000,-9.750000e+05,16.000000,0.000000,0.000000,1.000000,0.000000,-3.250000e+05,16.000000
25%,4.000000,2.000000,0.000000,2.000000,0.000000,0.000000e+00,51.000000,2.000000,8.333333,2.333333,0.000000,8.333333e+01,51.000000
50%,8.000000,2.000000,24.000000,3.000000,0.000000,3.000000e+02,58.000000,2.333333,28.000000,3.333333,0.000000,1.070000e+03,58.000000
75%,13.000000,3.000000,49.000000,5.000000,1.000000,4.100000e+03,64.000000,3.333333,43.333333,4.666667,0.333333,8.616667e+03,64.000000
max,30.000000,10.000000,168.000000,13.000000,1.000000,8.001500e+06,89.000000,9.666667,119.000000,10.000000,1.000000,2.672550e+06,89.000000


In [18]:
y=Xc['cesd']
X=Xc[['child','hrsusu','hsize','retired','wealth']]
model=lmp.FirstDifferenceOLS(y,X)
fd=model.fit(cov_type="robust")
print(fd)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                   cesd   R-squared:                        0.0021
Estimator:         FirstDifferenceOLS   R-squared (Between):             -0.0225
No. Observations:                6704   R-squared (Within):               0.0023
Date:                Tue, Oct 04 2022   R-squared (Overall):             -0.0197
Time:                        15:44:50   Log-likelihood                -2.168e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      2.8530
Entities:                        3353   P-value                           0.0141
Avg Obs:                       2.9994   Distribution:                  F(5,6699)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             2.8457
                            

## Pooled OLS

We can use statsmodels to estimate a simple OLS regression to explain the average drivers of the firm's demand for workers (Pooled OLS in panel data).

In [142]:
y=Xc['workers']
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
X=sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                workers   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     6745.
Date:                Tue, 27 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:50:40   Log-Likelihood:                -48280.
No. Observations:               39104   AIC:                         9.657e+04
Df Residuals:                   39097   BIC:                         9.663e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.2938      0.022    150.362      0.0

In [143]:
model=lmp.PooledOLS(y,X)
OLS=model.fit(cov_type="robust")
print(OLS)

                          PooledOLS Estimation Summary                          
Dep. Variable:                workers   R-squared:                        0.5086
Estimator:                  PooledOLS   R-squared (Between):              0.5474
No. Observations:               39104   R-squared (Within):               0.0917
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5086
Time:                        16:50:44   Log-likelihood                -4.828e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      6745.2
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,39097)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             7415.2
                            

## First differences 

In [144]:
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
model=lmp.FirstDifferenceOLS(y,X)
fd=model.fit(cov_type="robust")
print(fd)

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\shared\utility.py:187: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_levels except for the argument 'levels' will be keyword-only
  df.index = df.index.set_levels(final_levels, [0, 1])


                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                workers   R-squared:                        0.2001
Estimator:         FirstDifferenceOLS   R-squared (Between):             -1.3115
No. Observations:               14188   R-squared (Within):               0.2040
Date:                Tue, Sep 27 2022   R-squared (Overall):             -1.1216
Time:                        16:50:48   Log-likelihood                -1.448e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      591.42
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,14182)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             381.83
                            

## Fixed Effects 

In [145]:
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
X=sm.add_constant(X)
model=lmp.PanelOLS(y,X, entity_effects=True)
fe=model.fit(cov_type="robust")
print(fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                workers   R-squared:                        0.2255
Estimator:                   PanelOLS   R-squared (Between):              0.2717
No. Observations:               39104   R-squared (Within):               0.2255
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.2902
Time:                        16:50:58   Log-likelihood                -1.435e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      726.34
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,14970)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             332.31
                            

## Random Effects


In [146]:
model=lmp.RandomEffects(y,X)
re=model.fit(cov_type="robust")
print(re)

                        RandomEffects Estimation Summary                        
Dep. Variable:                workers   R-squared:                        0.3878
Estimator:              RandomEffects   R-squared (Between):              0.5399
No. Observations:               39104   R-squared (Within):               0.1514
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5017
Time:                        16:51:05   Log-likelihood                -3.453e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      4127.1
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,39097)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             5240.3
                            

In [147]:
re.variance_decomposition

Effects                   0.311160
Residual                  0.318662
Percent due to Effects    0.494045
Name: Variance Decomposition, dtype: float64

## Model comparison

In [148]:
print(lmp.compare({"FE": fe, "RE": re, "Pooled": OLS}))

                           Model Comparison                           
                                    FE                RE        Pooled
----------------------------------------------------------------------
Dep. Variable                  workers           workers       workers
Estimator                     PanelOLS     RandomEffects     PooledOLS
No. Observations                 39104             39104         39104
Cov. Est.                       Robust            Robust        Robust
R-squared                       0.2255            0.3878        0.5086
R-Squared (Within)              0.2255            0.1514        0.0917
R-Squared (Between)             0.2717            0.5399        0.5474
R-Squared (Overall)             0.2902            0.5017        0.5086
F-statistic                     726.34            4127.1        6745.2
P-value (F-stat)                0.0000            0.0000        0.0000
=====================     ============   ===============   ===========
const 

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\results.py:1022: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  1,


In [149]:
import numpy.linalg as la
from scipy import stats

def hausman(fe, re):
 diff = fe.params-re.params
 psi = fe.cov - re.cov
 dof = diff.size -1
 W = diff.dot(la.inv(psi)).dot(diff)
 pval = stats.chi2.sf(W, dof)
 return W, dof, pval

In [150]:
htest = hausman(fe, re) 
print("Hausman Test: chi-2 = {0}, df = {1}, p-value = {2}".format(htest[0], htest[1], htest[2]))

Hausman Test: chi-2 = 489.87847525273105, df = 6, p-value = 1.2731322706600914e-102


## Correlated Random Effects

In [155]:
X=Xc[['age','sales','fomento','export','tamano','impuestos','mage','msales','mfomento','mexport','mtamano','mimpuestos']]
X=sm.add_constant(X)
model=lmp.RandomEffects(y,X)
cre=model.fit(cov_type="robust")
print(cre)

                        RandomEffects Estimation Summary                        
Dep. Variable:                workers   R-squared:                        0.4211
Estimator:              RandomEffects   R-squared (Between):              0.5540
No. Observations:               39104   R-squared (Within):               0.2255
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5278
Time:                        16:53:07   Log-likelihood                -3.343e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      2369.9
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                F(12,39091)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             3144.8
                            

In [156]:
print(lmp.compare({"FE": fe, "RE": re, "CRE": cre}))

                             Model Comparison                             
                                    FE                RE               CRE
--------------------------------------------------------------------------
Dep. Variable                  workers           workers           workers
Estimator                     PanelOLS     RandomEffects     RandomEffects
No. Observations                 39104             39104             39104
Cov. Est.                       Robust            Robust            Robust
R-squared                       0.2255            0.3878            0.4211
R-Squared (Within)              0.2255            0.1514            0.2255
R-Squared (Between)             0.2717            0.5399            0.5540
R-Squared (Overall)             0.2902            0.5017            0.5278
F-statistic                     726.34            4127.1            2369.9
P-value (F-stat)                0.0000            0.0000            0.0000
=====================    

<font size="3">**Tarea 2**</font>

<u> *Instrucciones* </u>

Los resultados de los ejericicios propuestos se deben entregar como un notebook por correo electronico a *juan.caro@uni.lu* el dia 3/10 hasta las 21:00. 

Es importante considerar que el código debe poder ejecutarse en cualquier computadora con la data original del repositorio. Recordar la convencion para el nombre de archivo ademas de incluir en su documento titulos y encabezados por seccion. La data a utilizar es **charls.csv**.

Las variables tienen la siguiente descripcion:

- INID: identificador unico
- wave: periodo de la encuesta (1-3)
- cesd: puntaje en la escala de salud mental (0-30)
- child: numero de hijos
- drinkly: bebio alcohol en el ultimo mes (binario)
- hrsusu: horas promedio trabajo semanal
- hsize: tamano del hogar
- intmonth: mes en que fue encuestado/a (1-12)
- married: si esta casado/a (binario)
- retired: si esta pensionado/a (binario)
- schadj: años de escolaridad
- urban: zona urbana (binario)
- wealth: riqueza neta (miles RMB)
- age: edad al entrar a la encuesta (no varia entre periodos)

Preguntas:

1. Cargar la base de datos *charls.csv* en el ambiente. Identifique los tipos de datos que se encuentran en la base, realice estadisticas descriptivas sobre las variables importantes (Hint: Revisar la distribuciones, datos faltantes, outliers, etc.) y limpie las variables cuando sea necesario. 

**R:** Como se discutio en clases, al exportar la data original a CSV se cortaron los ID unicos de una cantidad importante de observaciones. Se sugirio trabajar solamente con los ID correctos, pero no se penalizo a nadie que utilizara la base completa (donde habian mas de 3 observaciones por ID lo que no es correcto dado que solo hay 3 periodos). Excluyendo las observaciones erroneas, la base final incluye aproximandamente 3,500 ID unicos. 

2. Ejecute un modelo Pooled OLS para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

**R:** En base al modelo Pooled, el indice CESD se incrementa con el numero de hijos, si el sexo es femenino, y disminuye en personas casadas y viviendo en zonas urbanas. Ademas el indice cae con el nivel de educacion, riqueza y el tamano del hogar. Aquellos retirados o trabajando mas horas a la semana muestran niveles de CESD mas altos. Sin embargo los resultados asumen que existe independencia entre las observaciones en el tiempo, lo cual no es correcto (por ej, un invididuo retirado en t=1 sigue retirado en t=2).

3. Ejecute un modelo de efectos fijos para explicar el puntaje en la escala de salud mental (CESD).  Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

**R:** El modelo FE comapara la variacion promedio de los individuos en el tiempo. En base a ello, los resultados muestran que el tamano del hogar, la situacion de retiro y las horas trabajadas no tienen efecto en el indice CESD, una vez considerado el sesgo por heterogeneidad fija en el tiempo. Por otro lado, un factor importante como el sexo, no puede ser estimado dado que es fijo en el tiempo.

4. Ejecute un modelo de efectos aleatorios para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

**R:** El modelo RE asume que la variacion fija en el tiempo (no observada) es independiente de cualquier factor observado, y procede a comparar la variacion entre individuos (no dentro de cada individuo). En este modelo, se observan cambios importantes en los coeficientes, tanto en significancia como en signo. Por ejemplo, la situacion de retiro es significativa, mientras que el estado civil ya no lo es. 

5. Comente los resultados obtenidos en 2, 3 y 4. ¿Cuáles y por qué existen las diferencias entre los resultados?. En su opinión, ¿Cuál sería el más adecuado para responder la pregunta de investgación y por qué? ¿Qué variables resultaron ser robustas a la especificación?

**R:** El test de Hausman sugiere que el modelo FE es mas apropiado para el analisis, sin embargo, como se discutio en clases, impide observar factores importantes que explican diferencias entre individuos como el sexo o estado civil, por tanto entrega una vision parcial (pero potencialmente insesgada) del analisis.

6. Ejecute un modelo de efectos aleatorios correlacionados (CRE) para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. Es este modelo adecuado, dada la data disponible, para modelar el componente no observado?

**R:** El modelo CRE usa las medias en el tiempo de las variables observadas para modelar el componente de heterogeneidad. Los resultados sugieren que las horas trabajas, entre otros factores, estan correlacionadas con la heterogeneidad no observada, y permiten identificar el modelo. Por construccion, los resultados son equivalentes a FE para los componentes que varian en el tiempo, pero ademas entrega estimacion de los componentes fijos (como edad y sexo).

7. Usando el modelo CRE, prediga la distribucion del componente no observado. Que puede inferir respecto de la heterogeneidad fija en el tiempo y su impacto en el puntaje CESD? 

**R:** Para predecir el componente no observado basta guardar los coeficientes y generar una variable que sea el producto de las variables promedio (con la letra m en el comienzo) por sus respectivos coeficientes. Este producto ponderado genera el valor estimado de la heterogeneidad fija en el tiempo y se puede plotear (scatter, por ejemplo) contra el puntaje CESD promedio en el tiempo. 

8. Usando sus respuestas anteriores, que modelo prefiere? que se puede inferir en general respecto del efecto de las variables explicativas sobre el puntaje CESD?

**R:** Comparando entre FE, RE y CRE, vemos que la salud mental via CESD tiene componentes importantes que varian en el tiempo (como el estado marital o el tamano del hogar), y otros fijos en el tiempo (como el sexo y la edad). Analizar entonces CESD require entender y separar elementos que describen la distribucion de la poblacion asi como los cambios individuales, y en particular tener en cuenta la heterogeneidad no observada, i.e. factores que afectan la salud mental de los individuos de forma fija en el tiempo pero que no estan disponibles en la data (e.g. genetica).
